In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import Angle, SkyCoord, EarthLocation, AltAz

import ipywidgets as widgets
from ipywidgets import interact, FloatSlider
from IPython.display import display, HTML
import pandas as pd 
import mircxpol as mp

In [2]:
path_csv = "./2022October_Wollaston_Reduced_NORMED AVERAGED/MIRCX_2022Oct19_W2S2_fancy.csv"
df = pd.read_csv(path_csv)

MJD_data = df['MJD'].values
HA_data = df[' HA'].values
# VisRatio
VisRatio_data = df[' VisRatio_normed'].values
VisRatio_err_data = df[' visratio_normed_err'].values
# PhaseDiff
PD_data = df[' PhaseDiff'].values
PD_err_data = df[' Phase_err'].values
# Tel1_ratio
T1_ratio_data = df[' Tel1_ratio'].values
T1_ratio_err_data = df['tel1_ratio_err'].values
# Tel2_ratio
T2_ratio_data = df['tel2_ratio'].values
T2_ratio_err_data = df['tel2_ratio_err'].values

MJD = Time(MJD_data, format='mjd')
tt = MJD.to_value('iso')

time = np.linspace(4.5, 10.01, len(HA_data)) * u.hour
ptime = Time("2022-10-19 0:00:00") + time 
HA = Angle(time).value
chara = EarthLocation.of_site("CHARA")
upsand = SkyCoord.from_name("ups and")
upsandaltaz = upsand.transform_to(AltAz(obstime = ptime, location=chara))
alt = upsandaltaz.alt
az = upsandaltaz.az
# Find the index of the maximum altitude
zenith_idx = np.argmax(alt)
zenith_time = HA[zenith_idx]
HA = HA - zenith_time

In [7]:
def plot_func(r_m31, c_m31, r_m41, c_m41, r_m81, c_m81, r_m32, c_m32, r_m42, c_m42, r_m82, c_m82, I12, Q12, U12, V12, Qii, Uii, Vii):

    var = np.array([r_m31, c_m31, r_m41, c_m41, r_m81, c_m81, r_m32, c_m32, r_m42, c_m42, r_m82, c_m82, I12, Q12, U12, V12, Qii, Uii, Vii])

    plt.figure(figsize=(16, 8))
    plt.subplot(2, 2, 1)
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.plot(HA.flatten(), mp.func1(alt, az, var, pol=True), linewidth=2, color='black')
    plt.plot(HA.flatten(), mp.func_comp(alt, az, var, pol=True)[0], label="VH", linestyle='--')
    plt.plot(HA.flatten(), mp.func_comp(alt, az, var, pol=True)[1], label="VV", linestyle='-.')
    plt.errorbar(HA_data, VisRatio_data, yerr = VisRatio_err_data, fmt = 'o', color = 'c')
    plt.ylabel("Norm Vis Ratio", fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)

    plt.subplot(2, 2, 2)
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.plot(HA, mp.func2(alt, az, var, pol=True), linewidth=2, color='black')
    plt.plot(HA.flatten(), mp.func_comp(alt, az, var, pol=True)[2], label="VH", linestyle='--')
    plt.plot(HA.flatten(), mp.func_comp(alt, az, var, pol=True)[3], label="VV", linestyle='-.')
    plt.errorbar(HA_data, PD_data, yerr = PD_err_data, fmt = 'o', color = 'c')
    plt.ylabel(r'$Differential~phase~(degrees)$', fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)


    plt.subplot(2, 2, 3)
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.plot(HA, mp.func3(alt, az, var, pol=True), linewidth=2, color='black')
    plt.errorbar(HA_data, T1_ratio_data, yerr = T1_ratio_err_data, fmt = 'o', color = 'c')
    plt.ylabel("Flux Ratio Tel1", fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)

    plt.subplot(2, 2, 4)
    plt.plot(HA, mp.func4(alt, az, var, pol=True), linewidth=2, color='black')
    plt.errorbar(HA_data, T2_ratio_data, yerr = T2_ratio_err_data, fmt = 'o', color = 'c')
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.ylabel("Flux Ratio Tel2", fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)

    plt.tight_layout()
    plt.show()
    


In [8]:
def main():

    initials = np.array([ 0.98974287,  0.06568035,  1.04895981,  0.00893904,  1.10702174,
       -0.37458241,  1.01377179, -0.06569218,  1.01210919,  0.42657349,
        1.09041511,  0.37458351])

    r_m31_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[0], description="M1-3", readout_format='.3f')
    c_m31_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[1], description="Phase M1-3", readout_format='.3f')
    r_m41_widget = FloatSlider(min=-2.100, max=2.000, step=0.001, value=initials[2], description = "M4-7", readout_format='.3f')
    c_m41_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[3], description = "Phase M4-7", readout_format='.3f')
    r_m81_widget = FloatSlider(min=-2.100, max=2.000, step=0.001, value=initials[4], description = "M8-19", readout_format='.3f')
    c_m81_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[5], description = "Phase M8-19", readout_format='.3f')

    r_m32_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[6], description = "M1-3", readout_format='.3f')
    c_m32_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[7], description = "Phase M1-3", readout_format='.3f')
    r_m42_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[8], description = "M4-7", readout_format='.3f')
    c_m42_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[9], description = "Phase M4-7", readout_format='.3f')
    r_m82_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[10], description = "M8-19", readout_format='.3f')
    c_m82_widget = FloatSlider(min=-2.000, max=2.000, step=0.001, value=initials[11], description = "Phase M8-19", readout_format='.3f')

    # r_m32_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M1-3", readout_format='.3f')
    # c_m32_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M1-3", readout_format='.3f')
    # r_m42_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M4-7", readout_format='.3f')
    # c_m42_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M4-7", readout_format='.3f')
    # r_m82_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M8-19", readout_format='.3f')
    # c_m82_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M8-19", readout_format='.3f')

    I12_widget = FloatSlider(min=0.000, max=1.000, step=0.001, value=1.00, description = "I12", readout_format='.3f')
    Q12_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Q12", readout_format='.3f')
    U12_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "U12", readout_format='.3f')
    V12_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "V12", readout_format='.3f')

    # Iii_widget = FloatSlider(min=0.000, max=1.000, step=0.001, value=1.00, description = "Iii", readout_format='.3f')
    Qii_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Qii", readout_format='.3f')
    Uii_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Uii", readout_format='.3f')
    Vii_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Vii", readout_format='.3f')

    left_title = widgets.HTML(value="<div style='text-align: center;'><h3>beam1</h3></div>")
    center_title = widgets.HTML(value="<div style='text-align: center;'><h3>beam2</h3></div>")
    right_title = widgets.HTML(value="<div style='text-align: center;'><h3>Stokes parameters</h3></div>")

    left_box = widgets.VBox([left_title, r_m31_widget, c_m31_widget, r_m41_widget, c_m41_widget, r_m81_widget, c_m81_widget])
    center_box = widgets.VBox([center_title, r_m32_widget, c_m32_widget, r_m42_widget, c_m42_widget, r_m82_widget, c_m82_widget])
    right_box = widgets.VBox([right_title, I12_widget, Q12_widget, U12_widget, V12_widget, Qii_widget, Uii_widget, Vii_widget]) # Iii_widget,
    all_widgets = widgets.HBox([left_box, center_box, right_box])

    out = widgets.interactive_output(plot_func, {'r_m31': r_m31_widget, 'c_m31': c_m31_widget, 'r_m41': r_m41_widget, 
                                                'c_m41': c_m41_widget, 'r_m81': r_m81_widget, 'c_m81': c_m81_widget, 
                                                'r_m32': r_m32_widget, 'c_m32': c_m32_widget, 'r_m42': r_m42_widget, 
                                                'c_m42': c_m42_widget, 'r_m82': r_m82_widget, 'c_m82': c_m82_widget,
                                                'I12': I12_widget, 'Q12': Q12_widget, 'U12': U12_widget, 'V12': V12_widget, 
                                                'Qii': Qii_widget, 'Uii': Uii_widget, 'Vii': Vii_widget})  # 'Iii': Iii_widget


    if I12_widget.value +Q12_widget.value > 1 - Qii_widget.value:
        print(f"I12 + Q12 is not within the expected range")

    elif I12_widget.value - Q12_widget.value > 1 - Qii_widget.value:
        print(f"I12 - Q12 is not within the expected range")

    css = """
    .bg-red {
        background-color: rgba(255, 0, 0, 0.25) !important; /* Adjust the last value (0.5) to change the transparency */
    }

    """

    # Create an HTML widget to apply CSS
    html = widgets.HTML(value="<style>{}</style>".format(css))

    # Function to update widget background color based on condition
    def update_color():
        if I12_widget.value +Q12_widget.value > 1 - Qii_widget.value:
            all_widgets.add_class("bg-red")
        elif I12_widget.value - Q12_widget.value > 1 - Qii_widget.value:
            all_widgets.add_class("bg-red")
        else:
            all_widgets.remove_class("bg-red")

    # Link widget value change to color update function
    I12_widget.observe(update_color, names='value')
    Q12_widget.observe(update_color, names='value')
    Qii_widget.observe(update_color, names='value')

    display(html)
    display(all_widgets, out)


if __name__ == "__main__":
    main()

HTML(value='<style>\n    .bg-red {\n        background-color: rgba(255, 0, 0, 0.25) !important; /* Adjust the …

Output()

In [5]:
np.sqrt(5) / (np.sqrt(10) + np.sqrt(41))

0.23376623391647994

In [6]:
np.sqrt(5) / np.sqrt(25 + 64)

0.23702273156998865

In [ ]:
    V_H = V_12_HH / (V_11_HH + V_22_HH)
    V_V = V_12_VV / (V_11_VV + V_22_VV)